In [1]:
using Revise

In [2]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using Gurobi
using Flux
using ProgressMeter
using MLDataPattern
using JLD2
using Plots; gr()

Plots.GRBackend()

In [3]:
import LCPSim
import LearningMPC
import BoxValkyries
reload("LearningMPC")
reload("BoxValkyries")

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/BoxValkyries.ji for module BoxValkyries.


In [4]:
robot = BoxValkyries.BoxValkyrie();
mvis = MechanismVisualizer(robot)
IJuliaCell(mvis)

Listening on 127.0.0.1:7000...
zmq_url=tcp://127.0.0.1:6000
web_url=http://127.0.0.1:7000/static/


MeshCat.IJuliaCell(MeshCat.ViewerWindow(ZMQ.Context(Ptr{Void} @0x000000000742a2a0, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x00000000073e6e10, Base.Filesystem._FDWatcher(Ptr{Void} @0x00000000073a6450, 58, (1, 0), Condition(Any[]), (false, false), 0))]), ZMQ.Socket(Ptr{Void} @0x00000000073e6e10, Base.Filesystem._FDWatcher(Ptr{Void} @0x00000000073a6450, 58, (1, 0), Condition(Any[]), (false, false), 0)), "http://127.0.0.1:7000/static/", "tcp://127.0.0.1:6000", MeshCat.ZMQServer.ZMQWebSocketBridge(ip"127.0.0.1", Set{WebSockets.WebSocket}(), MeshCat.SceneTrees.SceneNode(Nullable{Array{UInt8,1}}(), Nullable{Array{UInt8,1}}(), Dict("meshcat"=>MeshCat.SceneTrees.SceneNode(#NULL, #NULL, Dict("robot"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x72, 0x6f, 0x62, 0x6f, 0x74], Dict("world"=>MeshCat.SceneTrees.SceneNode(#NULL, #NULL, Dict("dummy"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x64, 0x75, 0x6d, 0x6d, 0x79], Dict("base"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x64, 0x75, 0x6d, 0x6d, 0x79, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict("core"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x2f, 0x63, 0x6f, 0x72, 0x65], Dict("rh_dummy"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x2f, 0x72, 0x68, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79], Dict("after_core_to_rh_rotation"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x5f, 0x72, 0x6f, 0x74, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict("geometry_2"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String,MeshCat.SceneTrees.SceneNode}()))),"rh"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x2f, 0x72, 0x68], Dict("after_core_to_rh_extension"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x65, 0x78, 0x74, 0x65, 0x6e, 0x73, 0x69, 0x6f, 0x6e], Dict("geometry_3"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String,MeshCat.SceneTrees.SceneNode}()))))))),"rf_dummy"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x2f, 0x72, 0x66, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79], Dict("after_core_to_rf_rotation"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x5f, 0x72, 0x6f, 0x74, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict("geometry_6"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x36], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x36], Dict{String,MeshCat.SceneTrees.SceneNode}()))),"rf"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x5f, 0x64, 0x75, 0x6d, 0x6d, 0x79, 0x2f, 0x72, 0x66], Dict("after_core_to_rf_extension"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x65, 0x78, 0x74, 0x65, 

In [5]:
function create_net()
    net = Chain(
        Dense(20, 32, elu),
        Dense(32, 32, elu),
        Dense(32, 10)
    )
    loss = (x, y) -> Flux.mse(net(x), y)
    net, loss
end

create_net (generic function with 1 method)

In [11]:
net, loss = create_net()
net_controller = x -> Flux.Tracker.data(net(state_vector(x)))
net_params = params(net)
optimizer = Flux.Optimise.ADADelta(net_params)

xstar = BoxValkyries.nominal_state(robot)

mpc_params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=10,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, 
        TimeLimit=60, 
        MIPGap=1e-1, 
        MIPGapAbs=5e-1,
        FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

Q, R = BoxValkyries.default_costs(robot)
feet = findbody.(robot.mechanism, ["lf", "rf"])
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, mpc_params.Δt, Point3D.(default_frame.(feet), 0., 0., 0.))
lqrsol.S .= 1 ./ mpc_params.Δt .* Q

mpc_controller = LearningMPC.MPCController(robot.mechanism, 
    robot.environment, mpc_params, lqrsol, 
    [net_controller, lqrsol]);

sample_sink = LearningMPC.MPCSampleSink{Float64}(true)
playback_sink = LearningMPC.PlaybackSink(mvis, mpc_params.Δt)

mpc_controller.callback = LearningMPC.call_each(
    sample_sink,
    playback_sink,
)

live_viewer = LearningMPC.live_viewer(mvis)


dagger_controller = LearningMPC.call_each(
    LearningMPC.dagger_controller(
        mpc_controller,
        net_controller,
        0.2),
    live_viewer
    )

termination = x -> false

dataset = LearningMPC.Dataset(lqrsol)

function collect_into!(data::Vector{<:LearningMPC.Sample})
    empty!(sample_sink)
    LearningMPC.randomize!(x0, xstar, 0.0, 1.5)
    results = LCPSim.simulate(x0, 
        dagger_controller,
        robot.environment, mpc_params.Δt, 30, 
        mpc_params.lcp_solver;
        termination=termination);
    append!(data, sample_sink.samples)
end

x0 = MechanismState{Float64}(robot.mechanism)

features(s::LearningMPC.Sample) = (s.state, s.uJ[:, 1])

Academic license - for non-commercial use only
Academic license - for non-commercial use only


features (generic function with 1 method)

In [12]:
datasets = Vector{LearningMPC.Dataset{Float64}}()
all_training_data = Vector{Tuple{Vector{Float64}, Vector{Float64}}}()
all_validation_data = Vector{Tuple{Vector{Float64}, Vector{Float64}}}()
losses = Vector{Tuple{Float64, Float64}}()

for i in 1:20
    dataset = LearningMPC.Dataset(lqrsol)
    for i in 1:2
        collect_into!(dataset.training_data)
    end
    collect_into!(dataset.testing_data)
    collect_into!(dataset.validation_data)
    append!(all_training_data, features.(dataset.training_data))
    append!(all_validation_data, features.(dataset.validation_data))
    
    @showprogress for i in 1:20
        Flux.train!(loss, shuffleobs(all_training_data), optimizer)
        push!(losses, 
            (mean(xy -> Flux.Tracker.data(loss(xy...)), 
                  all_training_data),
             mean(xy -> Flux.Tracker.data(loss(xy...)), 
                  all_validation_data)))
    end
    push!(datasets, dataset)
    
    jldopen("box-val.jld2", "w") do file
        file["datasets"] = datasets
        file["net"] = net
        file["lqrsol"] = lqrsol
        file["mpc_params"] = Dict(
            "Δt" => mpc_params.Δt,
            "horizon" => mpc_params.horizon,
        )
        file["all_training_data"] = all_training_data
        file["all_validation_data"] = all_validation_data
        file["losses"] = losses
    end
    
    plt = plot(first.(losses), label="training")
    plot!(plt, last.(losses), label="validation")
    ylims!(plt, (0, ylims(plt)[2]))
    display(plt)
end

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 
 
 
 
 
 training 
 
 
 
 validation

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
<polyline clip-path="url(#clip7002)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 29.1292,11.811 43.2622,18.4026 57.3951,24.9285 71.5281,31.4058 85.6611,37.9224 99.794,44.5825 113.927,51.36 128.06,58.3259 142.193,65.5571 156.326,73.0295 
 170.459,80.7803 184.592,88.8382 198.725,97.2442 212.858,105.969 226.991,115.061 241.124,124.545 255.257,134.323 269.39,144.456 283.523,154.867 297.656,165.547 
 311.789,250.479 325.922,261.927 340.055,272.45 354.187,282.329 368.32,291.065 382.453,298.655 396.586,305.093 410.719,310.747 424.852,315.167 438.985,318.648 
 453.118,321.454 467.251,323.59 481.384,325.271 495.517,326.59 509.65,327.659 523.783,328.527 537.916,329.295 552.049,329.984 566.182,330.604 580.315,331.155 
 
 "/>
<polyline clip-path="url(#clip7002)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 29.1292,87.288 43.2622,92.5697 57.3951,97.8587 71.5281,103.14 85.6611,108.475 99.794,113.949 113.927,119.534 128.06,125.309 142.193,131.317 156.326,137.544 
 170.459,144.018 184.592,150.753 198.725,157.772 212.858,165.054 226.991,172.643 241.124,180.566 255.257,188.746 269.39,197.226 283.523,205.935 297.656,214.852 
 311.789,242.838 325.922,256.526 340.055,269.295 354.187,281.459 368.32,292.411 382.453,302.133 396.586,310.587 410.719,318.251 424.852,324.394 438.985,329.327 
 453.118,333.425 467.251,336.552 481.384,339.018 495.517,340.897 509.65,342.353 523.783,343.39 537.916,344.199 552.049,344.843 566.182,345.328 580.315,345.669 
 
 "/>
 
 
 
 
 training 
 
 
 
 validation

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


captured: ErrorException("Unrecognized solution status: loaded")
captured: ErrorException("Unrecognized solution status: loaded")
captured: ErrorException("Unrecognized solution status: loaded")


...

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress:  90%|█████████████████████████████████████    |  ETA: 0:00:00

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 250 
 
 
 300 
 
 
 350 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
<polyline clip-path="url(#clip0402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 29.1292,11.811 30.5835,18.4026 32.0378,24.9285 33.4921,31.4058 34.9465,37.9224 36.4008,44.5825 37.8551,51.36 39.3094,58.3259 40.7637,65.5571 42.218,73.0295 
 43.6724,80.7803 45.1267,88.8382 46.581,97.2442 48.0353,105.969 49.4896,115.061 50.9439,124.545 52.3983,134.323 53.8526,144.456 55.3069,154.867 56.7612,165.547 
 58.2155,250.479 59.6698,261.927 61.1242,272.45 62.5785,282.329 64.0328,291.065 65.4871,298.655 66.9414,305.093 68.3957,310.747 69.85,315.167 71.3044,318.648 
 72.7587,321.454 74.213,323.59 75.6673,325.271 77.1216,326.59 78.5759,327.659 80.0303,328.527 81.4846,329.295 82.9389,329.984 84.3932,330.604 85.8475,331.155 
 87.3018,297.586 88.7562,299.888 90.2105,301.62 91.6648,303.175 93.1191,304.43 94.5734,305.533 96.0277,306.571 97.4821,307.513 98.9364,308.378 100.391,309.239 
 101.845,310.078 103.299,310.885 104.754,311.646 106.208,312.396 107.662,313.142 109.117,313.84 110.571,314.542 112.025,315.222 113.48,315.887 114.934,316.526 
 116.388,315.776 117.842,317.19 119.297,318.292 120.751,319.219 122.205,320.054 123.66,320.822 125.114,321.529 126.568,322.206 128.023,322.848 129.477,323.478 
 130.931,324.065 132.386,324.628 133.84,325.152 135.294,325.648 136.749,326.136 138.203,326.605 139.657,327.05 141.112,327.479 142.566,327.893 144.02,328.314 
 145.474,328.348 146.929,328.906 148.383,329.391 149.837,329.804 151.292,330.204 152.746,330.586 154.2,330.936 155.655,331.274 157.109,331.592 158.563,331.922 
 160.018,332.244 161.472,332.54 162.926,332.845 164.381,333.145 165.835,333.428 167.289,333.705 168.744,333.968 170.198,334.24 171.652,334.508 173.106,334.776 
 174.561,336.363 176.015,336.664 177.469,336.936 178.924,337.21 180.378,337.484 181.832,337.726 183.287,338.004 184.741,338.253 186.195,338.505 187.65,338.745 
 189.104,338.981 190.558,339.224 192.013,339.461 193.467,339.703 194.921,339.923 196.376,340.158 197.83,340.386 199.284,340.588 200.738,340.824 202.193,341.069 
 203.647,334.903 205.101,335.281 206.556,335.643 208.01,335.975 209.464,336.274 210.919,336.575 212.373,336.854 213.827,337.131 215.282,337.429 216.736,337.693 
 218.19,337.97 219.645,338.229 221.099,338.466 222.553,338.734 224.008,339 225.462,339.277 226.916,339.521 228.37,339.781 229.825,340.028 231.279,340.273 
 232.733,341.362 234.188,341.649 235.642,341.914 237.096,342.147 238.551,342.414 240.005,342.613 241.459,342.885 242.914,343.13 244.368,343.364 245.822,343.594 
 247.277,343.827 248.731,344.067 250.185,344.278 251.64,344.503 253.094,344.719 254.548,344.971 256.002,345.162 257.457,345.414 258.911,345.646 260.365,345.848 
 261.82,334.327 263.274,334.772 264.728,335.148 266.183,335.475 267.637,335.808 269.091,336.136 270.546,336.424 272,336.724 273.454,336.995 274.909,337.298 
 276.363,337.576 277.817,337.836 279.272,338.103 280.726,338.379 282.18,338.586 283.634,338.9 285.089,339.152 286.543,339.428 287.997,339.683 289.452,339.914 
 290.906,341.497 292.36,341.784 293.815,342.067 295.269,342.338 296.723,342.608 298.178,342.846 299.632,343.12 301.086,343.379 302.541,343.644 303.995,343.877 
 305.449,344.141 306.904,344.4 308.358,344.615 309.812,344.89 311.267,345.131 312.721,345.352 314.175,345.597 315.629,345.828 317.084,346.076 318.538,346.28 
 319.992,344.256 321.447,344.549 322.901,344.867 324.355,345.143 325.81,345.414 327.264,345.649 328.718,345.921 330.173,346.169 331.627,346.401 333.081,346.619 
 334.536,346.84 335.99,347.039 337.444,347.194 338.899,347.376 340.353,347.686 341.807,347.879 343.261,348.08 344.716,348.267 346.17,348.447 347.624,348.634 
 349.079,348.955 350.533,349.075 351.987,349.275 353.442,349.462 354.896,349.623 356.35,349.839 357.805,349.949 359.259,350.1

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress:  80%|█████████████████████████████████        |  ETA: 0:00:00

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
<polyline clip-path="url(#clip0602)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 29.1292,11.811 30.5106,18.4026 31.892,24.9285 33.2735,31.4058 34.6549,37.9224 36.0363,44.5825 37.4177,51.36 38.7991,58.3259 40.1805,65.5571 41.562,73.0295 
 42.9434,80.7803 44.3248,88.8382 45.7062,97.2442 47.0876,105.969 48.4691,115.061 49.8505,124.545 51.2319,134.323 52.6133,144.456 53.9947,154.867 55.3761,165.547 
 56.7576,250.479 58.139,261.927 59.5204,272.45 60.9018,282.329 62.2832,291.065 63.6646,298.655 65.0461,305.093 66.4275,310.747 67.8089,315.167 69.1903,318.648 
 70.5717,321.454 71.9532,323.59 73.3346,325.271 74.716,326.59 76.0974,327.659 77.4788,328.527 78.8602,329.295 80.2417,329.984 81.6231,330.604 83.0045,331.155 
 84.3859,297.586 85.7673,299.888 87.1488,301.62 88.5302,303.175 89.9116,304.43 91.293,305.533 92.6744,306.571 94.0558,307.513 95.4373,308.378 96.8187,309.239 
 98.2001,310.078 99.5815,310.885 100.963,311.646 102.344,312.396 103.726,313.142 105.107,313.84 106.489,314.542 107.87,315.222 109.251,315.887 110.633,316.526 
 112.014,315.776 113.396,317.19 114.777,318.292 116.159,319.219 117.54,320.054 118.921,320.822 120.303,321.529 121.684,322.206 123.066,322.848 124.447,323.478 
 125.828,324.065 127.21,324.628 128.591,325.152 129.973,325.648 131.354,326.136 132.736,326.605 134.117,327.05 135.498,327.479 136.88,327.893 138.261,328.314 
 139.643,328.348 141.024,328.906 142.405,329.391 143.787,329.804 145.168,330.204 146.55,330.586 147.931,330.936 149.313,331.274 150.694,331.592 152.075,331.922 
 153.457,332.244 154.838,332.54 156.22,332.845 157.601,333.145 158.982,333.428 160.364,333.705 161.745,333.968 163.127,334.24 164.508,334.508 165.89,334.776 
 167.271,336.363 168.652,336.664 170.034,336.936 171.415,337.21 172.797,337.484 174.178,337.726 175.56,338.004 176.941,338.253 178.322,338.505 179.704,338.745 
 181.085,338.981 182.467,339.224 183.848,339.461 185.229,339.703 186.611,339.923 187.992,340.158 189.374,340.386 190.755,340.588 192.137,340.824 193.518,341.069 
 194.899,334.903 196.281,335.281 197.662,335.643 199.044,335.975 200.425,336.274 201.806,336.575 203.188,336.854 204.569,337.131 205.951,337.429 207.332,337.693 
 208.714,337.97 210.095,338.229 211.476,338.466 212.858,338.734 214.239,339 215.621,339.277 217.002,339.521 218.383,339.781 219.765,340.028 221.146,340.273 
 222.528,341.362 223.909,341.649 225.291,341.914 226.672,342.147 228.053,342.414 229.435,342.613 230.816,342.885 232.198,343.13 233.579,343.364 234.96,343.594 
 236.342,343.827 237.723,344.067 239.105,344.278 240.486,344.503 241.868,344.719 243.249,344.971 244.63,345.162 246.012,345.414 247.393,345.646 248.775,345.848 
 250.156,334.327 251.537,334.772 252.919,335.148 254.3,335.475 255.682,335.808 257.063,336.136 258.445,336.424 259.826,336.724 261.207,336.995 262.589,337.298 
 263.97,337.576 265.352,337.836 266.733,338.103 268.115,338.379 269.496,338.586 270.877,338.9 272.259,339.152 273.64,339.428 275.022,339.683 276.403,339.914 
 277.784,341.497 279.166,341.784 280.547,342.067 281.929,342.338 283.31,342.608 284.692,342.846 286.073,343.12 287.454,343.379 288.836,343.644 290.217,343.877 
 291.599,344.141 292.98,344.4 294.361,344.615 295.743,344.89 297.124,345.131 298.506,345.352 299.887,345.597 301.269,345.828 302.65,346.076 304.031,346.28 
 305.413,344.256 306.794,344.549 308.176,344.867 309.557,345.143 310.938,345.414 312.32,345.649 313.701,345.921 315.083,346.169 316.464,346.401 317.846,346.619 
 319.227,346.84 320.608,347.039 321.99,347.194 323.371,347.376 324.753,347.686 326.134,347.879 327.515,348.08 328.897,348.267 330.278,348.447 331.66,348.634 
 333.041,348.955 334.423,349.075 335.804,349.275 337.185,349.462 338.567,349.623 339.948,349.839 341.33,349.949 342.711,350.108 344.092,350.308 345.474,350.499 
 34

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


In [41]:
x0 = MechanismState{Float64}(robot.mechanism)
LearningMPC.randomize!(x0, xstar, 0.5, 2.0)
results = LCPSim.simulate(x0, 
    net_controller,
    robot.environment, mpc_params.Δt, 100, 
    mpc_params.lcp_solver,
    termination=termination);
LearningMPC.playback(mvis, results, 0.05)

In [ ]:
LearningMPC.playback(mvis, results, 0.05)

In [28]:
    @showprogress for i in 1:100
        Flux.train!(loss, shuffleobs(all_training_data), optimizer)
        push!(losses, 
            (mean(xy -> Flux.Tracker.data(loss(xy...)), 
                  all_training_data),
             mean(xy -> Flux.Tracker.data(loss(xy...)), 
                  all_validation_data)))
    end
    plt = plot(first.(losses), label="training")
    plot!(plt, last.(losses), label="validation")
    ylims!(plt, (0, ylims(plt)[2]))
    plt

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 600 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
<polyline clip-path="url(#clip1002)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 29.1292,11.811 30.0494,18.4026 30.9696,24.9285 31.8897,31.4058 32.8099,37.9224 33.7301,44.5825 34.6503,51.36 35.5704,58.3259 36.4906,65.5571 37.4108,73.0295 
 38.331,80.7803 39.2511,88.8382 40.1713,97.2442 41.0915,105.969 42.0117,115.061 42.9318,124.545 43.852,134.323 44.7722,144.456 45.6924,154.867 46.6126,165.547 
 47.5327,250.479 48.4529,261.927 49.3731,272.45 50.2933,282.329 51.2134,291.065 52.1336,298.655 53.0538,305.093 53.974,310.747 54.8941,315.167 55.8143,318.648 
 56.7345,321.454 57.6547,323.59 58.5749,325.271 59.495,326.59 60.4152,327.659 61.3354,328.527 62.2556,329.295 63.1757,329.984 64.0959,330.604 65.0161,331.155 
 65.9363,297.586 66.8564,299.888 67.7766,301.62 68.6968,303.175 69.617,304.43 70.5371,305.533 71.4573,306.571 72.3775,307.513 73.2977,308.378 74.2179,309.239 
 75.138,310.078 76.0582,310.885 76.9784,311.646 77.8986,312.396 78.8187,313.142 79.7389,313.84 80.6591,314.542 81.5793,315.222 82.4994,315.887 83.4196,316.526 
 84.3398,315.776 85.26,317.19 86.1801,318.292 87.1003,319.219 88.0205,320.054 88.9407,320.822 89.8609,321.529 90.781,322.206 91.7012,322.848 92.6214,323.478 
 93.5416,324.065 94.4617,324.628 95.3819,325.152 96.3021,325.648 97.2223,326.136 98.1424,326.605 99.0626,327.05 99.9828,327.479 100.903,327.893 101.823,328.314 
 102.743,328.348 103.664,328.906 104.584,329.391 105.504,329.804 106.424,330.204 107.344,330.586 108.264,330.936 109.185,331.274 110.105,331.592 111.025,331.922 
 111.945,332.244 112.865,332.54 113.785,332.845 114.706,333.145 115.626,333.428 116.546,333.705 117.466,333.968 118.386,334.24 119.307,334.508 120.227,334.776 
 121.147,336.363 122.067,336.664 122.987,336.936 123.907,337.21 124.828,337.484 125.748,337.726 126.668,338.004 127.588,338.253 128.508,338.505 129.428,338.745 
 130.349,338.981 131.269,339.224 132.189,339.461 133.109,339.703 134.029,339.923 134.95,340.158 135.87,340.386 136.79,340.588 137.71,340.824 138.63,341.069 
 139.55,334.903 140.471,335.281 141.391,335.643 142.311,335.975 143.231,336.274 144.151,336.575 145.071,336.854 145.992,337.131 146.912,337.429 147.832,337.693 
 148.752,337.97 149.672,338.229 150.593,338.466 151.513,338.734 152.433,339 153.353,339.277 154.273,339.521 155.193,339.781 156.114,340.028 157.034,340.273 
 157.954,341.362 158.874,341.649 159.794,341.914 160.714,342.147 161.635,342.414 162.555,342.613 163.475,342.885 164.395,343.13 165.315,343.364 166.236,343.594 
 167.156,343.827 168.076,344.067 168.996,344.278 169.916,344.503 170.836,344.719 171.757,344.971 172.677,345.162 173.597,345.414 174.517,345.646 175.437,345.848 
 176.357,334.327 177.278,334.772 178.198,335.148 179.118,335.475 180.038,335.808 180.958,336.136 181.879,336.424 182.799,336.724 183.719,336.995 184.639,337.298 
 185.559,337.576 186.479,337.836 187.4,338.103 188.32,338.379 189.24,338.586 190.16,338.9 191.08,339.152 192,339.428 192.921,339.683 193.841,339.914 
 194.761,341.497 195.681,341.784 196.601,342.067 197.522,342.338 198.442,342.608 199.362,342.846 200.282,343.12 201.202,343.379 202.122,343.644 203.043,343.877 
 203.963,344.141 204.883,344.4 205.803,344.615 206.723,344.89 207.643,345.131 208.564,345.352 209.484,345.597 210.404,345.828 211.324,346.076 212.244,346.28 
 213.165,344.256 214.085,344.549 215.005,344.867 215.925,345.143 216.845,345.414 217.765,345.649 218.686,345.921 219.606,346.169 220.526,346.401 221.446,346.619 
 222.366,346.84 223.286,347.039 224.207,347.194 225.127,347.376 226.047,347.686 226.967,347.879 227.887,348.08 228.808,348.267 229.728,348.447 230.648,348.634 
 231.568,348.955 232.488,349.075 233.408,349.275 234.329,349.462 235.249,349.623 236.169,349.839 237.089,349.949 238.009,350.108 238.929